In [ ]:
import mlflow

import pandas as pd
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression , Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope




Run following command in terminal

python -m mlflow server --backend-store-uri sqlite:///backend.db

In [ ]:
mlflow.set_tracking_uri("http://127.0.0.1:5000") 
mlflow.set_experiment("Electricity Demand Prediction")

In [ ]:
def read_dataframe(filename):
    
    """
    Reads a CSV file into a DataFrame, processes date and demand columns, and adds additional features.

    :param filename: Path to the CSV file to be read.
    :return: DataFrame with processed columns and additional features (year, month, day, hour, day_of_week, is_weekend, holiday).
    """
    
    df = pd.read_csv(filename)
    df['date'] = pd.to_datetime(df['date'])
    df['demand'] = pd.to_numeric(df['demand'], errors='coerce').astype('float')
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    df['hr'] = df['date'].dt.hour
    df['day_of_week'] = df['date'].dt.dayofweek  # Monday=0, Sunday=6
    df['is_weekend'] = df['date'].dt.dayofweek >= 5  # True if weekend, False otherwise
    
    holidays = calendar().holidays(start=df['date'].min(), end=df['date'].max())
    df['holiday'] = df['date'].isin(holidays).astype(int)
    #display(df.head())
    #print(df.dtypes)
    
    
    return df

In [ ]:
dataset_path = "dataset.csv"

In [ ]:
df = read_dataframe(dataset_path)

In [ ]:
# Define features and target
X = df.drop('demand', axis=1)
X = X.drop('date', axis=1)

y = df['demand']

In [ ]:
# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)

In [ ]:
def objective(params):
    """
    Trains an XGBoost model with specified parameters and logs metrics using MLflow.

    :param params: Dictionary of hyperparameters for the XGBoost model.
    :return: Dictionary containing the loss ('rmse') and the status of the optimization.
    """
    with mlflow.start_run():
        mlflow.set_tag("model", "xgboost")
        mlflow.log_params(params)
        mlflow.xgboost.autolog()
        booster = xgb.train(
            params=params,
            dtrain=train,
            num_boost_round=1000,
            evals=[(valid, 'validation')],
            early_stopping_rounds=50
        )
        y_pred = booster.predict(valid)
        rmse = mean_squared_error(y_val, y_pred, squared=False)
        mlflow.log_metric("rmse", rmse)

    return {'loss': rmse, 'status': STATUS_OK}

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)
print(best_result)

In [ ]:
new_observation = pd.DataFrame({
    'temperature' : [21],
    'year': [2024],
    'month': [8],
    'day': [12],
    'hr': [3],
    'day_of_week': [1],
    'is_weekend': [0],
    'holiday': [0]
})

In [ ]:
import mlflow
logged_model = 'runs:/0ac14477b69842e195919d1cab69d066/model'

# Load model as a PyFuncModel
loaded_model = mlflow.pyfunc.load_model(logged_model)


In [ ]:
loaded_model.predict(pd.DataFrame(new_observation))

In [ ]:
print(best_result)